In [1]:
import os
import sys
sys.path.append('..')

In [2]:
import random
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
from source.data import dataLoader
from source.model import Model
from source.loss import Loss, Metric
from source.train import trainModel

### Seed

In [4]:
seed = 2017
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

### Model

In [5]:
model = Model()
model = model.to('cuda:0')

### Loss

In [6]:
loss = Loss()
metric = Metric() 

### Optimizer

In [7]:
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
schedular = ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

### Data

In [8]:
train = pd.read_csv('../../data/train.csv')
valid = pd.read_csv('../../data/valid.csv')
path = '../../data/'
train, valid = dataLoader(path, train, valid)
train = DataLoader(train, batch_size=4, shuffle=True, num_workers=1, drop_last=True)
valid = DataLoader(valid, batch_size=4, shuffle=True, num_workers=1, drop_last=True)

Train Images: 2976 Valid Images: 454


### Train

In [9]:
trainer = {}
trainer['model'] = model
trainer['train_data'] = train
trainer['valid_data'] = valid
trainer['loss_fn'] = loss
trainer['metric_fn'] = metric
trainer['optimizer'] = optimizer
trainer['save_path'] = '../../data/model.pt'
trainer['epochs'] = 10
trainer['batch'] = 4
trainer['scheduler'] = schedular

In [10]:
trainModel(**trainer)

100% 2976/2976 [06:19<00:00,  7.85it/s, trn_ls=0.83998, trn_mt=0.13873, val_ls=0.78066, val_mt=0.16482]
100% 2976/2976 [06:13<00:00,  7.97it/s, trn_ls=0.75851, trn_mt=0.17840, val_ls=0.75005, val_mt=0.18298]
100% 2976/2976 [06:12<00:00,  7.98it/s, trn_ls=0.74349, trn_mt=0.18779, val_ls=0.74289, val_mt=0.19026]
100% 2976/2976 [06:13<00:00,  7.96it/s, trn_ls=0.73241, trn_mt=0.19551, val_ls=0.72027, val_mt=0.20325]
100% 2976/2976 [06:13<00:00,  7.98it/s, trn_ls=0.71824, trn_mt=0.20451, val_ls=0.71235, val_mt=0.20875]
100% 2976/2976 [06:12<00:00,  7.98it/s, trn_ls=0.70494, trn_mt=0.21377, val_ls=0.70988, val_mt=0.20959]
100% 2976/2976 [06:12<00:00,  7.98it/s, trn_ls=0.69638, trn_mt=0.21973, val_ls=0.69512, val_mt=0.21960]
100% 2976/2976 [06:12<00:00,  7.98it/s, trn_ls=0.68890, trn_mt=0.22481, val_ls=0.70093, val_mt=0.21628]
100% 2976/2976 [06:12<00:00,  8.00it/s, trn_ls=0.68000, trn_mt=0.23090, val_ls=0.69897, val_mt=0.21935]
100% 2976/2976 [06:11<00:00,  8.01it/s, trn_ls=0.67760, trn_mt=0